# Adding Housing Construction Materials

Based on a discussion on the performance of the model without the construction materials features, we are testing out using the materials but converting from municipality to grid values based on weights computed from Google Building Footprint data. Will look at the performance of the model after adding these.

In [1]:
%load_ext jupyter_black
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os

In [16]:
base_dir = Path(os.getenv("STORM_DATA_DIR")) / "analysis/02_new_model_input/"
input_dir = base_dir / "05_vulnerablility/input/"
output_dir = base_dir / "05_vulnerablility/output/"
weights_dir = base_dir / "02_housing_damage/input/Google Footprint Data/"
hu_dir = base_dir / "02_housing_damage/output/"

In [17]:
mun_to_grid_weights = pd.read_csv(weights_dir / "ggl_mun_to_grid_weights.csv")
construction_materials_df = pd.read_excel(
    input_dir
    / "180814_construction-materials-of-the-outer-walls-and-roof_by-city_municipality.xlsx",
    sheet_name="by category",
)
hu_bygrid = pd.read_csv(hu_dir / "transformed_housingunits_bygrid.csv")

In [18]:
construction_materials_df.columns

Index(['Region', 'Region Code', 'Province', 'Province Code',
       'Municipality_City', 'Municipality_City Code', 'Housing Units',
       'Strong Roof/Strong Wall', 'Strong Roof/Light Wall',
       'Strong Roof/Salvage Wall', 'Light Roof/Strong Wall',
       'Light Roof/Light Wall', 'Light Roof/Salvage Wall',
       'Salvaged Roof/Strong Wall', 'Salvaged Roof/Light Wall',
       'Salvaged Roof/Salvage Wall'],
      dtype='object')

In [35]:
construction_materials_df["Strong Roof/Strong Wall"].sum()

14153053

In [19]:
merged_df = construction_materials_df.merge(
    mun_to_grid_weights[["ADM3_PCODE", "id", "Centroid", "weight"]],
    how="right",
    left_on="Municipality_City Code",
    right_on="ADM3_PCODE",
)

In [20]:
# There are 2 ways of going about this conversion to grid values
# 1. Convert the mun values to grid values using the weights and compute the % from the grid HUs
# 2. Compute % values by municipality then convert those values using the weights to grid values.

In [21]:
# Method 1
# merged_df.loc[:, merged_df.columns.str.contains("Roof")]
merged_df.drop(
    [
        "Region",
        "Region Code",
        "Province",
        "Province Code",
        "Municipality_City",
        "Housing Units",
    ],
    inplace=True,
    axis=1,
)

In [30]:
merged_df[
    merged_df.columns[merged_df.columns.str.contains("Roof")]
] = merged_df.loc[:, merged_df.columns.str.contains("Roof")].multiply(
    merged_df["weight"], axis="index"
)

In [31]:
merged_grid_df = merged_df.groupby(["id", "Centroid"]).sum().reset_index()

C:\Users\pauni\AppData\Local\Temp\ipykernel_27268\1508198058.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  merged_grid_df = merged_df.groupby(["id", "Centroid"]).sum().reset_index()


In [34]:
merged_grid_df["Strong Roof/Strong Wall"].sum()

14153052.999999996

In [38]:
merged_grid_df.drop("weight", inplace=True, axis=1)

In [46]:
hu_conmat_df = hu_bygrid.merge(merged_grid_df, on=["id", "Centroid"])

In [48]:
hu_conmat_df[
    hu_conmat_df.columns[hu_conmat_df.columns.str.contains("Roof")]
] = (
    hu_conmat_df.loc[:, hu_conmat_df.columns.str.contains("Roof")].divide(
        hu_conmat_df["hu_bygrid"], axis="index"
    )
) * 100

In [51]:
hu_conmat_df.drop(
    ["numbuildings", "Number of Household"], inplace=True, axis=1
)

In [52]:
# writing to csv
hu_conmat_df.to_csv(
    output_dir / "construction_materials_bygrid.csv", index=False
)

In [ ]:
# Method 2

